In [26]:
import torch

模拟x输入

In [27]:
seq_size=10
dim_size=32
batch_size=2

x=torch.arange(0,batch_size*seq_size*dim_size*2,step=2).view(batch_size,dim_size,seq_size).type(torch.float64).requires_grad_(True)
x.shape

torch.Size([2, 32, 10])

In [28]:
target_seq_size=15

Pytorch官方插值

In [29]:
y=torch.nn.functional.interpolate(x,size=target_seq_size,mode='linear',align_corners=True)
print(f'第0维特征:{y[0][0]}\n第1维特征:{y[0][1]}')

第0维特征:tensor([ 0.0000,  1.2857,  2.5714,  3.8571,  5.1429,  6.4286,  7.7143,  9.0000,
        10.2857, 11.5714, 12.8571, 14.1429, 15.4286, 16.7143, 18.0000],
       dtype=torch.float64, grad_fn=<SelectBackward0>)
第1维特征:tensor([20.0000, 21.2857, 22.5714, 23.8571, 25.1429, 26.4286, 27.7143, 29.0000,
        30.2857, 31.5714, 32.8571, 34.1429, 35.4286, 36.7143, 38.0000],
       dtype=torch.float64, grad_fn=<SelectBackward0>)


自实现插值

In [30]:
def linear_interp(x,target_seq_size):
    i_new=torch.linspace(0,x.shape[2]-1,steps=target_seq_size)
    print(f'i_new:{i_new}') # (target_seq_size,)
    i_new_left=i_new.view(1,1,target_seq_size).expand(x.size(0),x.size(1),-1).floor().long()
    print(f'i_new_left[0][0]:{i_new_left[0][0]}')
    i_new_right=torch.clamp(i_new_left+1,0,x.size(2)-1)
    print(f'i_new_right[0][0]:{i_new_right[0][0]}')

    x_new_left=torch.gather(x,dim=2,index=i_new_left)
    x_new_right=torch.gather(x,dim=2,index=i_new_right)
    print(f'x_new_left[0][0]:{x_new_left[0][0]}\nx_new_right[0][0]:{x_new_right[0][0]}')

    i_new_weight=i_new.view(1,1,target_seq_size).expand(x.size(0),x.size(1),-1)-i_new_left
    x_new=x_new_left+(x_new_right-x_new_left)*i_new_weight
    return x_new

In [31]:
x_new=linear_interp(x,target_seq_size)
x_new_torch=torch.nn.functional.interpolate(x,size=(target_seq_size,),mode='linear',align_corners=True)
print(f'【测试结论】\n第0维特征:{x_new[0][0]}\n第1维特征:{x_new[0][1]}\n与torch官方一致性:{torch.all(torch.abs(x_new-x_new_torch)<1e-5)}')

i_new:tensor([0.0000, 0.6429, 1.2857, 1.9286, 2.5714, 3.2143, 3.8571, 4.5000, 5.1429,
        5.7857, 6.4286, 7.0714, 7.7143, 8.3571, 9.0000])
i_new_left[0][0]:tensor([0, 0, 1, 1, 2, 3, 3, 4, 5, 5, 6, 7, 7, 8, 9])
i_new_right[0][0]:tensor([1, 1, 2, 2, 3, 4, 4, 5, 6, 6, 7, 8, 8, 9, 9])
x_new_left[0][0]:tensor([ 0.,  0.,  2.,  2.,  4.,  6.,  6.,  8., 10., 10., 12., 14., 14., 16.,
        18.], dtype=torch.float64, grad_fn=<SelectBackward0>)
x_new_right[0][0]:tensor([ 2.,  2.,  4.,  4.,  6.,  8.,  8., 10., 12., 12., 14., 16., 16., 18.,
        18.], dtype=torch.float64, grad_fn=<SelectBackward0>)
【测试结论】
第0维特征:tensor([ 0.0000,  1.2857,  2.5714,  3.8571,  5.1429,  6.4286,  7.7143,  9.0000,
        10.2857, 11.5714, 12.8571, 14.1429, 15.4286, 16.7143, 18.0000],
       dtype=torch.float64, grad_fn=<SelectBackward0>)
第1维特征:tensor([20.0000, 21.2857, 22.5714, 23.8571, 25.1429, 26.4286, 27.7143, 29.0000,
        30.2857, 31.5714, 32.8571, 34.1429, 35.4286, 36.7143, 38.0000],
       dtype=torch.fl

In [32]:
from torchviz import make_dot
dot=make_dot(x_new,params={'x':x,'x_new':x_new})
dot.render("linearinterp",format="png")

'linearinterp.png'